In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#%run -i '../input/utilexternal/misc.py'
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#!pip install import-ipynb

In [ ]:
import sys
sys.path.insert(1, '/kaggle/input/cifarnetworks')

In [ ]:
import cv2
import math
import torch
import heapq
import random
import argparse
import numpy as np
import torchvision
import pandas as pd
import torch.nn as nn
import warnings
from PIL import Image
from skimage import io
from pathlib import Path
from preactresnetb import *
#from resnet50 import *
#from wideresnet import *
#from function_1 import *
#from function_4_2 import *
#from utils.conf import *
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import (Dataset, DataLoader,random_split,)
import os, glob
from torchvision.io import read_image, ImageReadMode
from collections import OrderedDict
import torch.nn.functional as F
from skimage.util import random_noise

In [ ]:
warnings.filterwarnings("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device in torch.cuda.is_available() {device}")
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [ ]:
model_options = ['preactresnet18', 'preactresnet34', 'preactresnet50', 'preactresnet101', 'wideresnet']
dataset_options = ['cifar10', 'cifar100', 'tinyimagenet', 'imagenet']
parser = argparse.ArgumentParser(description='ACResizeMix')
parser.add_argument('--dataset', '-d', default='cifar10', choices=dataset_options)
parser.add_argument('--model', '-a', default='preactresnet18', choices=model_options)
parser.add_argument('--batch_size', type=int, default=64, help='input batch size for training (default:128)')
parser.add_argument('--epochs', type=int, default=300, help='number of epochs to train (default: 200)')
parser.add_argument('--lr', type=float, default=0.05, help='learning rate')
parser.add_argument('--data_augmentation', action='store_true', default=True, help='augment data by flipping and cropping')
parser.add_argument('--no-cuda', action='store_true', default=False, help='enables CUDA training')
parser.add_argument('--acresize_prob', default=0.5, type=float, help='ACResizeMix probability')
args, _ = parser.parse_known_args()

In [ ]:

cudnn.benchmark = True  # Should make training should go faster for large models

seed = 1
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
test_id = args.dataset + '_' + args.model
print(args)
if args.dataset == 'cifar10':
    mean = [0.4914, 0.4822, 0.4465]
    std = [0.2470, 0.2435, 0.2616]
elif args.dataset == 'cifar100':
    mean = [0.5071, 0.4865, 0.4409]
    std = [0.2673, 0.2564, 0.2762]
#transform_train, transform_test = augmentation(mean, std)

In [ ]:
def augmentation(mean, std):
    transform_train = transforms.Compose([
        #transforms.ToPILImage(),
        transforms.RandomCrop(32, padding=4),
        #transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        #transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
        #transforms.RandomPerspective(distortion_scale=0.3, p=0.5),
        #transforms.RandomPosterize(bits=2, p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        ])
    return transform_train, transform_test

In [ ]:
transform_train, transform_test = augmentation(mean, std)

In [ ]:

if args.dataset == 'cifar10':
    num_classes = 10
    train_dataset = datasets.CIFAR10(root='data/',train=True,transform=transform_train,download=True)
    test_dataset = datasets.CIFAR10(root='data/', train=False,transform=transform_test,download=True)
elif args.dataset == 'cifar100':
    num_classes = 100
    train_dataset = datasets.CIFAR100(root='data/', train=True, transform=transform_train, download=True)
    test_dataset = datasets.CIFAR100(root='data/',  train=False, transform=transform_test, download=True)
elif args.dataset == 'tinyimagenet':
    num_classes = 200
    train_dataset = datasets.CIFAR100(root='data/', train=True, transform=transform_train, download=True)
    test_dataset = datasets.CIFAR100(root='data/',  train=False, transform=transform_test, download=True)
elif args.dataset == 'imagenet':
    num_classes = 1000
    train_dataset = datasets.CIFAR100(root='data/', train=True, transform=transform_train, download=True)
    test_dataset = datasets.CIFAR100(root='data/',  train=False, transform=transform_test, download=True)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,  batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=args.batch_size, shuffle=False,  pin_memory=True,  num_workers=2)

In [ ]:
def show_batch(dl):
    import matplotlib.pyplot as plt
    from torchvision.utils import make_grid
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(10,10))
        #images = denormalize(images, *data_statistics)
        ax.imshow(make_grid(images, 10).permute(1,2,0))
        break
#show_batch(train_loader)

In [ ]:
if args.model == 'preactresnet18':
    cnn = PreActResNet(PreActBlock, [2, 2, 2, 2], num_classes=num_classes)
elif args.model == 'preactresnet34':
    cnn = PreActResNet(PreActBlock, [3,4,6,3], num_classes=num_classes)
elif args.model == 'preactresnet50':
    cnn = PreActResNet(PreActBottleneck, [3,4,6,3], num_classes=num_classes)
elif args.model == 'preactresnet101':
    cnn = PreActResNet(PreActBottleneck, [3,4,23,3], num_classes=num_classes)
elif args.model == 'wideresnet':
    cnn = WideResNet(depth=28, num_classes=num_classes, widen_factor=10, dropRate=0.0)

cnn = cnn.to(device)
#cnn = nn.DataParallel(cnn).to(device)

In [ ]:
#print(f"cnn {cnn}")

In [ ]:
class model_bn(nn.Module):
    def __init__(self, feature_size=512,classes_num=10):

        super(model_bn, self).__init__()

        self.features = nn.Sequential(*list(cnn.children())[:-2])
        #print(f"self.features {self.features}")

        #self.max = nn.MaxPool2d(kernel_size=4, stride=4)
        self.max = nn.AvgPool2d(kernel_size=4, stride=4)

        self.num_ftrs = 512*1*1
        self.classifier = nn.Sequential(OrderedDict([
              #('bn1', nn.BatchNorm1d(feature_size)),
              #('rel', nn.ReLU(inplace=True)),
              ('fc1', nn.Linear(feature_size, classes_num)),
              ]))

    def forward(self, x):

        feats = self.features(x)
        x = self.max(feats)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x, feats

In [ ]:
cnn = model_bn(512, 10)
cnn = cnn.to(device)
cudnn.benchmark = True

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='mean').to(device)
cnn_optimizer = torch.optim.SGD(cnn.parameters(), lr=args.lr, momentum=0.9, nesterov=True, weight_decay=5e-4)
scheduler = MultiStepLR(cnn_optimizer, milestones=[60, 120, 180, 240], gamma=0.2, verbose=False)

In [ ]:
class Loss:
    def Error(pred, target):
        sample = len(target)
        pred = torch.exp(pred)
        pred = pred/torch.sum(pred, axis=1, keepdims=True)
        if len(target.shape)==1:
            confidence = pred[range(sample), target]
        elif len(target.shape)==2:
            confidence = sum(pred*target, axis=1)
        eror = -torch.log10(confidence)
        total = torch.mean(eror)
        return total
    
def cutmix_criterion(criterion, pred, y_a, y_b, lam, lum):
    return lam * criterion(pred, y_a) + lum* criterion(pred, y_b)

In [ ]:
load_model = False
if load_model:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('/kaggle/input/checkpoint/'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('/kaggle/input/checkpoint/ckpt.pth')
    cnn.load_state_dict(checkpoint['cnn'])
    cnn_optimizer.load_state_dict(checkpoint['cnn_optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])
    best_acc = checkpoint['acc']
    print(f"best_acc {best_acc}")
    start_epoch = checkpoint['epoch']
    print(f"start_epoch {start_epoch}")
    start_epoch = start_epoch + 1
    print(f"start_epoch {start_epoch}")

In [ ]:
def show_batch_1(images_c, labels_c):
    import matplotlib.pyplot as plt
    j=0
    from torchvision.utils import make_grid
    for i, (image, labels) in enumerate(zip(images_c, labels_c)):
        j=j+1
        #print(f"images.shape show_batch {images.shape}")
        #print(f"i {i}")  
        
        fig, ax = plt.subplots(figsize=(2, 2))
        ax.imshow(make_grid(image.cpu(), 10).permute(1,2,0))
        if j ==128:
            break

In [ ]:
def cutmix(data, targets, alpha=1.0):
    """Applique CutMix sur un batch de données"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = data.size(0)
    index = torch.randperm(batch_size).cuda()

    # Sélection des coordonnées du rectangle à couper
    H, W = data.shape[2], data.shape[3]
    r_x = np.random.randint(W)
    r_y = np.random.randint(H)
    r_w = int(W * np.sqrt(1 - lam))
    r_h = int(H * np.sqrt(1 - lam))

    x1 = np.clip(r_x - r_w // 2, 0, W)
    x2 = np.clip(r_x + r_w // 2, 0, W)
    y1 = np.clip(r_y - r_h // 2, 0, H)
    y2 = np.clip(r_y + r_h // 2, 0, H)

    # Appliquer CutMix
    data[:, :, y1:y2, x1:x2] = data[index, :, y1:y2, x1:x2]
    
    # Ajustement du label
    lam, lum = 1 - ((x2 - x1) * (y2 - y1) / (W * H)), (x2 - x1) * (y2 - y1) / (W * H)
    targets_a, targets_b = targets, targets[index]
    return data, targets_a, targets_b, lam, lum

In [ ]:
# Train the model
total_step = len(train_loader)
def train(epoch):
    cnn.train()
    losses = []
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        sal = np.random.beta(1.0, 1.0)
        if sal <=0.8 and epoch >=10:  
            wfmaps,_ = get_spm(cnn, images, labels)
            if epoch >=300:
                images, labels = maxgrad_show(cnn, images, labels, wfmaps, 32)
                break

            images, labels_a, labels_b, lam, lum = cutmix(images, labels)
            #break
            #show_batch_1(images, labels_a)
            #break
            cnn.zero_grad()
            outputs, _ = cnn(images)
            #loss = Loss.Error(outputs, labels_a)*lam + Loss.Error(outputs, labels_b)*lum
            loss = cutmix_criterion(criterion, outputs, labels_a, labels_b, lam, lum)
            #loss = criterion(outputs, labels_a)*lam + criterion(outputs, labels_b)*lum
            loss = loss.mean()
            #loss = loss/2
            losses.append(loss.item())
        else:
            # Forward pass
            #show_batch_1(images, labels_a)
            #break
            cnn.zero_grad()
            outputs, _ = cnn(images)
            loss = criterion(outputs, labels)
            #loss = Loss.Error(outputs, labels)
            losses.append(loss.item())
            # Backward and optimize
        loss.backward()
        cnn_optimizer.step()
    losses = sum(losses)/len(losses)
    print(f"TrainEpoch Loss LR  {epoch+1} {losses} {cnn_optimizer.param_groups[0]['lr']}")

In [ ]:
# Test the model
def test(epoch):
    global best_acc
    cnn.eval()
    correct = 0
    total = 0
    losses = []
    #with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs, _ = cnn(images)
        loss = criterion(outputs, labels)
        losses.append(loss.item())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    losses = sum(losses)/len(losses)
    print(f"Epoch Loss LR Accuracy {epoch+1} {losses} {cnn_optimizer.param_groups[0]['lr']} {100.*(correct / total)}")

    # Save checkpoint.
    acc = 100.*correct/total
    if epoch >= 2:
        #if acc > best_acc:
        #print('Saving..')
        state = {
            'cnn': cnn.state_dict(),
            'cnn_optimizer' : cnn_optimizer.state_dict(),
            'scheduler' : scheduler.state_dict(),
            'acc': acc,
            'lr': cnn_optimizer.param_groups[0]['lr'],
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc
    scheduler.step(epoch)

In [ ]:
for epoch in range(start_epoch, start_epoch+302):

    train(epoch)
    if epoch % 40 ==0 and epoch <= 250:
        test(epoch)
    elif epoch > 250:
        test(epoch)